# Universidad Nacional de Chimborazo

**Facultad:** Facultad de Ingeniería  
**Carrera:** Ciencia de Datos e IA  
**Materia:** Modelamiento  
**Docente:** Estalin Mejia H.  
**Semestre:** Tercero

---

## Trabajo Autónomo 2

### Análisis de Regresión Lineal Múltiple: Predicción de Arrestos en Crímenes de Chicago

**Autores:**  
- Alisson Atupaña
- Mario Camacho
- Lenin Lopez

**Fecha:** 24 de octubre de 2025

**Riobamba - Ecuador**

---


In [3]:
# Cargar las librerías necesarias
library(readr)
library(dplyr)

# Establecer el directorio de trabajo
setwd("C:/Users/mario/OneDrive/Documentos/Universidad/Tercer semestre/Modelamiento/Autonomos/Autonomo_2/R_Notebooks/archive")

# Listar todos los archivos CSV en el directorio
archivos_csv <- list.files(pattern = "*.csv", full.names = TRUE)
print("Archivos CSV encontrados:")
print(archivos_csv)

[1] "Archivos CSV encontrados:"
[1] "./yellow_tripdata_2015-01.csv" "./yellow_tripdata_2016-01.csv"
[3] "./yellow_tripdata_2016-02.csv" "./yellow_tripdata_2016-03.csv"
[1] "./yellow_tripdata_2015-01.csv" "./yellow_tripdata_2016-01.csv"
[3] "./yellow_tripdata_2016-02.csv" "./yellow_tripdata_2016-03.csv"


In [4]:
# Leer todos los archivos CSV y almacenar sus nombres de columnas
lista_dataframes <- list()
todas_las_columnas <- c()

for (i in 1:length(archivos_csv)) {
  archivo <- archivos_csv[i]
  print(paste("Leyendo archivo:", basename(archivo)))
  
  # Leer el archivo CSV
  df <- read_csv(archivo, show_col_types = FALSE)
  
  # Guardar el dataframe en la lista
  lista_dataframes[[basename(archivo)]] <- df
  
  # Obtener nombres de columnas y agregarlas al vector total
  columnas_archivo <- names(df)
  todas_las_columnas <- c(todas_las_columnas, columnas_archivo)
  
  print(paste("Columnas en", basename(archivo), ":", length(columnas_archivo)))
  print(columnas_archivo)
  print("---")
}

[1] "Leyendo archivo: yellow_tripdata_2015-01.csv"
[1] "Columnas en yellow_tripdata_2015-01.csv : 19"
 [1] "VendorID"              "tpep_pickup_datetime"  "tpep_dropoff_datetime"
 [4] "passenger_count"       "trip_distance"         "pickup_longitude"     
 [7] "pickup_latitude"       "RateCodeID"            "store_and_fwd_flag"   
[10] "dropoff_longitude"     "dropoff_latitude"      "payment_type"         
[13] "fare_amount"           "extra"                 "mta_tax"              
[16] "tip_amount"            "tolls_amount"          "improvement_surcharge"
[19] "total_amount"         
[1] "---"
[1] "Leyendo archivo: yellow_tripdata_2016-01.csv"
[1] "Columnas en yellow_tripdata_2016-01.csv : 19"
 [1] "VendorID"              "tpep_pickup_datetime"  "tpep_dropoff_datetime"
 [4] "passenger_count"       "trip_distance"         "pickup_longitude"     
 [7] "pickup_latitude"       "RatecodeID"            "store_and_fwd_flag"   
[10] "dropoff_longitude"     "dropoff_latitude"      "payment_ty

In [7]:
# Obtener columnas únicas de todos los datasets
columnas_unicas <- unique(todas_las_columnas)

# Mostrar resultados
print("=== RESUMEN DE ANÁLISIS ===")
print(paste("Total de archivos CSV procesados:", length(archivos_csv)))
print(paste("Total de columnas (incluyendo duplicados):", length(todas_las_columnas)))
print(paste("Total de columnas ÚNICAS en la unión:", length(columnas_unicas)))
print("")
print("Columnas únicas encontradas:")
print(columnas_unicas)

[1] "=== RESUMEN DE ANÁLISIS ==="


[1] "Total de archivos CSV procesados: 4"
[1] "Total de columnas (incluyendo duplicados): 76"
[1] "Total de columnas ÚNICAS en la unión: 20"
[1] ""
[1] "Columnas únicas encontradas:"
 [1] "VendorID"              "tpep_pickup_datetime"  "tpep_dropoff_datetime"
 [4] "passenger_count"       "trip_distance"         "pickup_longitude"     
 [7] "pickup_latitude"       "RateCodeID"            "store_and_fwd_flag"   
[10] "dropoff_longitude"     "dropoff_latitude"      "payment_type"         
[13] "fare_amount"           "extra"                 "mta_tax"              
[16] "tip_amount"            "tolls_amount"          "improvement_surcharge"
[19] "total_amount"          "RatecodeID"           


In [8]:
# Análisis adicional: mostrar qué columnas aparecen en cada archivo
print("=== ANÁLISIS DETALLADO POR ARCHIVO ===")
for (nombre_archivo in names(lista_dataframes)) {
  df <- lista_dataframes[[nombre_archivo]]
  print(paste("Archivo:", nombre_archivo))
  print(paste("Filas:", nrow(df)))
  print(paste("Columnas:", ncol(df)))
  print("Nombres de columnas:")
  print(names(df))
  print("---")
}

[1] "=== ANÁLISIS DETALLADO POR ARCHIVO ==="


[1] "Archivo: yellow_tripdata_2015-01.csv"
[1] "Filas: 12748986"
[1] "Columnas: 19"
[1] "Nombres de columnas:"
 [1] "VendorID"              "tpep_pickup_datetime"  "tpep_dropoff_datetime"
 [4] "passenger_count"       "trip_distance"         "pickup_longitude"     
 [7] "pickup_latitude"       "RateCodeID"            "store_and_fwd_flag"   
[10] "dropoff_longitude"     "dropoff_latitude"      "payment_type"         
[13] "fare_amount"           "extra"                 "mta_tax"              
[16] "tip_amount"            "tolls_amount"          "improvement_surcharge"
[19] "total_amount"         
[1] "---"
[1] "Archivo: yellow_tripdata_2016-01.csv"
[1] "Filas: 10906858"
[1] "Columnas: 19"
[1] "Nombres de columnas:"
 [1] "VendorID"              "tpep_pickup_datetime"  "tpep_dropoff_datetime"
 [4] "passenger_count"       "trip_distance"         "pickup_longitude"     
 [7] "pickup_latitude"       "RatecodeID"            "store_and_fwd_flag"   
[10] "dropoff_longitude"     "dropoff_latitude

In [9]:
# Calcular el total de filas (datos) de todos los datasets
total_filas <- 0

print("=== CONTEO TOTAL DE DATOS ===")
for (nombre_archivo in names(lista_dataframes)) {
  df <- lista_dataframes[[nombre_archivo]]
  filas_archivo <- nrow(df)
  total_filas <- total_filas + filas_archivo
  print(paste("Archivo:", nombre_archivo, "- Filas:", filas_archivo))
}

print("---")
print(paste("TOTAL DE DATOS (filas) en todos los datasets:", total_filas))
print(paste("TOTAL DE ARCHIVOS:", length(lista_dataframes)))
print(paste("PROMEDIO DE FILAS POR ARCHIVO:", round(total_filas / length(lista_dataframes), 2)))

[1] "=== CONTEO TOTAL DE DATOS ==="
[1] "Archivo: yellow_tripdata_2015-01.csv - Filas: 12748986"
[1] "Archivo: yellow_tripdata_2016-01.csv - Filas: 10906858"
[1] "Archivo: yellow_tripdata_2016-02.csv - Filas: 11382049"
[1] "Archivo: yellow_tripdata_2016-03.csv - Filas: 12210952"
[1] "---"
[1] "TOTAL DE DATOS (filas) en todos los datasets: 47248845"
[1] "TOTAL DE ARCHIVOS: 4"
[1] "PROMEDIO DE FILAS POR ARCHIVO: 11812211.25"


In [10]:
# Unir todos los dataframes en uno solo
df_unido <- bind_rows(lista_dataframes, .id = "archivo_origen")

# Guardar el dataframe unido como CSV
write_csv(df_unido, "datos_unidos.csv")

# Guardar el dataframe unido como RDS
saveRDS(df_unido, "datos_unidos.rds")

print("Archivos 'datos_unidos.csv' y 'datos_unidos.rds' guardados exitosamente.")

[1] "Archivos 'datos_unidos.csv' y 'datos_unidos.rds' guardados exitosamente."
